## Handling Missing Data in Pandas

The difference between data found in many tutorials and data in the real world is that
real-world data is rarely clean and homogeneous. In particular, many interesting
datasets will have some amount of data missing. To make matters even more complicated,
different data sources may indicate missing data in different ways.

#### Missing Data in Pandas
The way in which Pandas handles missing values is constrained by its reliance on the
NumPy package, which does not have a built-in notion of NA values for nonfloating-
point data types.

### None: Pythonic missing data
The first sentinel value used by Pandas is None, a Python singleton object that is often
used for missing data in Python code. Because None is a Python object, it cannot be
used in any arbitrary NumPy/Pandas array, but only in arrays with data type
'object' (i.e., arrays of Python objects):

In [3]:
import pandas as pd 
import numpy as np 

In [4]:
vals1 = np.array([1, 2, None, 3, 4])
vals1

array([1, 2, None, 3, 4], dtype=object)

This dtype=object means that the best common type representation NumPy could
infer for the contents of the array is that they are Python objects. While this kind of
object array is useful for some purposes, any operations on the data will be done at
the Python level, with much more overhead than the typically fast operations seen for
arrays with native types:

In [5]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
%timeit np.arange(1E6, dtype=dtype).sum()
print()

dtype = object
dtype = int
2.91 ms ± 545 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



The use of Python objects in an array also means that if you perform aggregations
like sum() or min() across an array with a None value, you will generally get an error: This reflects the fact that addition between an integer and None is undefined.

### NaN: Missing numerical data
The other missing data representation, NaN (acronym for Not a Number), is different;
it is a special floating-point value recognized by all systems that use the standard
IEEE floating-point representation:

In [9]:
vals2 = np.array([1, 2, np.nan, 3, 4])
print(vals2.dtype)
vals2

float64


array([ 1.,  2., nan,  3.,  4.])

Notice that NumPy chose a native floating-point type for this array: this means that
unlike the object array from before, this array supports fast operations pushed into
compiled code. You should be aware that NaN is a bit like a data virus—it infects any
other object it touches. Regardless of the operation, the result of arithmetic with NaN
will be another NaN:

In [11]:
print(1 + np.nan)
print(22 ** np.nan)

nan
nan


Note that this means that aggregates over the values are well defined (i.e., they don’t
result in an error) but not always useful:

In [14]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:

In [15]:
# NumPy does provide some special aggregations that will ignore these missing values:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(10.0, 1.0, 4.0)

Keep in mind that NaN is specifically a floating-point value; there is no equivalent
NaN value for integers, strings, or other types.

## NaN and None in Pandas
NaN and None both have their place, and Pandas is built to handle the two of them
nearly interchangeably, converting between them where appropriate:

In [19]:
pd.Series([1, 2, np.nan, 3, 4, None])

0    1.0
1    2.0
2    NaN
3    3.0
4    4.0
5    NaN
dtype: float64

For types that don’t have an available sentinel value, Pandas automatically type-casts
when NA values are present. For example, if we set a value in an integer array to
np.nan, it will automatically be upcast to a floating-point type to accommodate the
NA:

In [20]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [21]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automatically
converts the None to a NaN value.

## Operating on Null Values
Pandas treats None and NaN as essentially interchangeable for indicating
missing or null values. To facilitate this convention, there are several useful
methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

isnull()
:Generate a Boolean mask indicating missing values

notnull()
:Opposite of isnull()

dropna()
:Return a filtered version of the data

fillna()
:Return a copy of the data with missing values filled or imputed

### Detecting null values
Pandas data structures have two useful methods for detecting null data: isnull() and
notnull(). Either one will return a Boolean mask over the data. For example:

In [22]:
data = pd.Series([1, 2, np.nan, None, 4, 5])
data

0    1.0
1    2.0
2    NaN
3    NaN
4    4.0
5    5.0
dtype: float64

In [26]:
print(data.isnull())
print(data.notnull())

0    False
1    False
2     True
3     True
4    False
5    False
dtype: bool
0     True
1     True
2    False
3    False
4     True
5     True
dtype: bool


In [29]:
# slicing off null values
data[data.notnull()]

0    1.0
1    2.0
4    4.0
5    5.0
dtype: float64

### Dropping null values
In addition to the masking used before, there are the convenience methods, dropna()
(which removes NA values) and fillna() (which fills in NA values). For a Series,
the result is straightforward:

In [30]:
data.dropna()

0    1.0
1    2.0
4    4.0
5    5.0
dtype: float64

For a DataFrame, there are more options. Consider the following DataFrame:

In [32]:
df = pd.DataFrame([[1, np.nan, 2],[2, 3, 5],[np.nan, 4, 6]], columns=list('ABC'))
df

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


We cannot drop single values from a DataFrame; we can only drop full rows or full
columns. Depending on the application, you might want one or the other, so
dropna() gives a number of options for a DataFrame.
By default, dropna() will drop all rows in which any null value is present:

In [34]:
# by default, dropna() drop all rows in which any null value is present
df.dropna()

,A,B,C
1,2.0,3.0,5


Alternatively, you can drop NA values along a different axis; axis=1 drops all columns
containing a null value:

In [38]:
# dropping NA values along the column axis
df.dropna(axis='columns')

,C
0,2
1,5
2,6


But this drops some good data as well; you might rather be interested in dropping
rows or columns with all NA values, or a majority of NA values. This can be specified
through the how or thresh parameters, which allow fine control of the number of
nulls to allow through.

The default is how='any', such that any row or column (depending on the axis keyword)
containing a null value will be dropped. You can also specify how='all', which
will only drop rows/columns that are all null values:

In [42]:
df[3] = np.nan
df

,A,B,C,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [44]:
# drop only rows/columns that are all null values
df.dropna(axis='columns', how='all')

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


For finer-grained control, the thresh parameter lets you specify a minimum number
of non-null values for the row/column to be kept:

In [47]:
df.dropna(axis='rows', thresh=3)

,A,B,C,3
1,2.0,3.0,5,NaN


Here the first and last row have been dropped, because they contain only two nonnull
values.

### Filling null values
Sometimes rather than dropping NA values, you’d rather replace them with a valid
value. This value might be a single number like zero, or it might be some sort of
imputation or interpolation from the good values. You could do this in-place using
the isnull() method as a mask, but because it is such a common operation Pandas
provides the fillna() method, which returns a copy of the array with the null values
replaced.

In [48]:
# Consider the following series
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [49]:
# We can fill NA entries with a single value, such as zero:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [50]:
# We can specify a forward-fill to propagate the previous value forward:
# forward fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [51]:
# Or we can specify a back-fill to propagate the next values backward:
# backward fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For DataFrames, the options are similar, but we can also specify an axis along which
the fills take place:

In [52]:
df

,A,B,C,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [54]:
# forward fill in DataFrame
df.fillna(method='ffill', axis=1)

,A,B,C,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Notice that if a previous value is not available during a forward fill, the NA value
remains.

In [57]:
# backward fill in DataFrame
df.fillna(method='bfill', axis=1)

,A,B,C,3
0,1.0,2.0,2.0,NaN
1,2.0,3.0,5.0,NaN
2,4.0,4.0,6.0,NaN
